## Final Proposal: Airbnb Case Study in New York City##
#### Principal Investigator: Nan Jiang <br> Email: [nancy.jiang@nyu.edu](nancy.jiang@nyu.edu)

This project will study how pricing of listed Airbnb accommodation within New York City is subject to location, room types and number of reviews as well as how it compares with neighboring hotels. Specifically, New York City, due to its enormous traffic flow and skyrocketing rental fees, accommocates a huge airbnb market across its 5 distinct boroughs. This project will group current Airbnb listings by roomtypes and boroughs and visualize the outstanding difference in prices.

The key element of the project is the use of [NYC Airbnb Listing Dataset](http://data.insideairbnb.com/united-states/ny/new-york-city/2018-03-04/visualisations/listings.csv) and [Yelp's API](https://api.yelp.com/v3/businesses/{id)  (This part I still need some time to figure out). The dataset is a thorough collection of data on prices, whereabouts and room type of each listing. While the latter provides access to price levels of some selection of hotels in the neighborhood, which gives a rough sense how airbnb prices would compare across. Details of this dataset are described below in the data report.

I anticipate that the project will have three sections.

- Basic statistics about the breakdown of room types and difference in average pricing across boroughs will be reported. What is the cheapest/most expensive combination of boroughs and room_type to stay at?

- An analysis on how Airbnb accommodation compares to neighboring hotels in terms of prices. Specifically, I would like to compute the average Airbnb price within each borough and compare it to average hotel price within the neignbourhood.

- An analysis on how occupancy rate differ due to number of reviews and timing of the year within the borough that has the most listings.

## Data Report ##

**Overview:** The data behind my project comes from the [Inside Airbnb](http://insideairbnb.com/). It is an independent, non-commercial set of data that aggregated all publicly avaliable information of Airbnb listings across the world. Their [NYC Airbnb Listing Dataset](http://data.insideairbnb.com/united-states/ny/new-york-city/2018-03-04/visualisations/listings.csv) provides access to key metrics and information including, prices, location, room type, number of reviews and dates of availability for each listing.

**Important Variables:** The key series of variables I will be focusing on in this project include the prices and neighborhood of the listing. 

In my report I will also try to get access to Yelp's API to understand average hotel prices within different neighborhoods. 

The *Geography* that I will work with is New York City. However, the same analysis could be easily applied to other cities as well.

**Access** I will use [Inside Airbnb](http://insideairbnb.com/) to download and access the core dataset. Below I demonstrate that I have the ability to access the data.

**Requisite Packages** Below I bring in the packages I need...

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import requests

%matplotlib inline

**Grabing the Data:** Below I use the Yelp API to grab prices and location information for hotels. First, I create xxx to pass to Yelp.

In [1]:
from yelpapi import YelpAPI

In [17]:
my_api_key = "V9KIR9KSlZgIunfSgwFNUY6DW1GBr-MJrwQJepeswI0ASq3koN6eWgijk2KywRxwbrecey"
my_api_key = my_api_key + "7O39IFr1DXXz-FnLjA844Wb99UKL4dh7XbCeWRiGUg_y2SOb1gyJ7mWnYx"

Y = YelpAPI(my_api_key)

type(Y)

yelpapi.yelpapi.YelpAPI

yelpapi.yelpapi.YelpAPI

In [120]:
url = "http://data.insideairbnb.com/united-states/ny/new-york-city/2018-03-04/visualisations/listings.csv"
ablist = pd.read_csv(url)

To clean up the dataset, I first removed all listings whose price is less than 10 USD, as according to [Airbnb](airbnb.com), 10 USD is the minimum starting price. 

In [126]:
ablist["price"] = pd.to_numeric(ablist["price"])
ablist = ablist[ablist.price >= 10]

I then picked out the columns of interest to me:

In [127]:
dfhost = ablist[["id","host_name"]]
dflist = ablist[["price","minimum_nights"]]
dfreview = ablist[["number_of_reviews","reviews_per_month"]]
dfavail = ablist [["availability_365"]]

By concatenating them together and setting index, I arrived at the cleaned up version dataframe I can readily use to create pivot tables and solve for section one. 

In [133]:
ablists = pd.concat([dfhost, dflist, dfreview, dfavail], axis=1, join_axes=[dfhost.index])
ablists = ablist.set_index(["neighbourhood_group","room_type"])
newab = ablists.sort_index(ascending = True)
newab

id  \
neighbourhood_group room_type                   
Bronx               Entire home/apt     63610   
                    Entire home/apt    149777   
                    Entire home/apt    182177   
                    Entire home/apt    385824   
                    Entire home/apt    449680   
                    Entire home/apt    525293   
                    Entire home/apt    755528   
                    Entire home/apt    791452   
                    Entire home/apt    958444   
                    Entire home/apt   1390964   
                    Entire home/apt   1487706   
                    Entire home/apt   1934765   
                    Entire home/apt   1936633   
                    Entire home/apt   2062630   
                    Entire home/apt   2296024   
                    Entire home/apt   2814931   
                    Entire home/apt   3185829   
                    Entire home/apt   3312276   
                    Entire home/apt   3628328   
                    Entire home/apt   3857687   
                    Entire home/apt   3953314   
                    Entire home/apt   4039789   
                    Entire home/apt   4216435   
                    Entire home/apt   4434797   
                    Entire home/apt   4449559   
                    Entire home/apt   4618235   
                    Entire home/apt   4667707   
                    Entire home/apt   4738429   
                    Entire home/apt   4963237   
                    Entire home/apt   4963511   
...                                       ...   
Staten Island       Private room     21013880   
                    Private room     21030003   
                    Private room     21062944   
                    Private room     21087187   
                    Private room     21350875   
                    Private room     21370266   
                    Private room     21501184   
                    Private room     21550437   
                    Private room     21623610   
                    Private room     21901025   
                    Private room     22161376   
                    Private room     22276159   
                    Private room     22420304   
                    Private room     22516765   
                    Private room     22772484   
                    Private room     22856025   
                    Private room     22937305   
                    Private room     22971554   
                    Private room     23290679   
                    Private room     23300336   
                    Private room     23308563   
                    Private room     23433585   
                    Private room     23446184   
                    Private room     23449778   
                    Private room     23465773   
                    Private room     23469673   
                    Shared room      18774943   
                    Shared room      18775025   
                    Shared room      22600184   
                    Shared room      22977021   

                                                                                  name  \
neighbourhood_group room_type                                                            
Bronx               Entire home/apt  DOMINIQUE'S NY CHIC SUITE-wifi/metro/sleeps 3*...   
                    Entire home/apt  Artsy, one bedroom 20 min from 42nd Grand Cent...   
                    Entire home/apt                PRIVATE FLAT / APARTMENT- $SPECIAL$   
                    Entire home/apt  New York City- Riverdale Modern two bedrooms unit   
                    Entire home/apt                   Artfully Decorated 2 Bedroom Apt   
                    Entire home/apt                                        Yankee Nest   
                    Entire home/apt                 PRIVATE BATH/TONS OF SUNLIGHT/SAFE   
                    Entire home/apt  condominium near Manhattan, free gas and electric   
                    Entire home/apt                Grea

**Problem: I tried using groupby, but had difficulty in converting the dataframegroupby object into a dataframe that I could work with**

In [131]:
ablists.groupby(["neighbourhood_group","room_type"]).count()

id   name  host_id  host_name  \
neighbourhood_group room_type                                           
Bronx               Entire home/apt    258    258      258        258   
                    Private room       581    578      581        579   
                    Shared room         37     36       37         37   
Brooklyn            Entire home/apt   9050   9050     9050       9036   
                    Private room     10956  10949    10956      10922   
                    Shared room        420    420      420        419   
Manhattan           Entire home/apt  12853  12844    12853      12830   
                    Private room      8721   8715     8721       8699   
                    Shared room        565    565      565        564   
Queens              Entire home/apt   1786   1786     1786       1785   
                    Private room      3081   3080     3081       3073   
                    Shared room        206    205      206        206   
Staten Island       Entire home/apt    136    136      136        136   
                    Private room       155    155      155        155   
                    Shared room          4      4        4          4   

                                     neighbourhood  latitude  longitude  \
neighbourhood_group room_type                                             
Bronx               Entire home/apt            258       258        258   
                    Private room               581       581        581   
                    Shared room                 37        37         37   
Brooklyn            Entire home/apt           9050      9050       9050   
                    Private room             10956     10956      10956   
                    Shared room                420       420        420   
Manhattan           Entire home/apt          12853     12853      12853   
                    Private room              8721      8721       8721   
                    Shared room                565       565        565   
Queens              Entire home/apt           1786      1786       1786   
                    Private room              3081      3081       3081   
                    Shared room                206       206        206   
Staten Island       Entire home/apt            136       136        136   
                    Private room               155       155        155   
                    Shared room                  4         4          4   

                                     price  minimum_nights  number_of_reviews  \
neighbourhood_group room_type                                                   
Bronx               Entire home/apt    258             258                258   
                    Private room       581             581                581   
                    Shared room         37              37                 37   
Brooklyn            Entire home/apt   9050            9050               9050   
                    Private room     10956           10956              10956   
                    Shared room        420             420                420   
Manhattan           Entire home/apt  12853           12853              12853   
                    Private room      8721            8721               8721   
                    Shared room        565             565                565   
Queens              Entire home/apt   1786            1786               1786   
                    Private room      3081            3081               3081   
                    Shared room        206             206                206   
Staten Island       Entire home/apt    136             136                136   
                    Private room       155             155                155   
                    Shared room          4               4                  4   

                                     last_review  reviews_per_month  \
neighbourhood_group room_type                                         
Br

As shown below, by manipulating the dataframe, I am able to quickly figure out relative popularity of the 5 boroughs in terms of airbnb listings.

In [113]:
cab = ablists.groupby(["neighbourhood_group"]).count()

In [114]:
cab

,id,host_name,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365
neighbourhood_group,,,,,,,
Bronx,876,874,876,876,876,666,876
Brooklyn,20426,20377,20426,20426,20426,15869,20426
Manhattan,22139,22093,22139,22139,22139,17228,22139
Queens,5073,5064,5073,5073,5073,3900,5073
Staten Island,295,295,295,295,295,215,295


Below is an initial attempt to figure out the neighborhoods with the highest amount of airbnb listings within each borough. Ultimately, I would like to map out the concentration of the airbnb listings in New York.

In [75]:
cab.id.sort_values(ascending = False)

neighbourhood_group  neighbourhood             
Brooklyn             Williamsburg                  4325
                     Bedford-Stuyvesant            3571
Manhattan            Harlem                        2884
Brooklyn             Bushwick                      2489
Manhattan            East Village                  2096
                     Upper West Side               2037
                     Hell's Kitchen                1869
                     Upper East Side               1783
Brooklyn             Crown Heights                 1687
Manhattan            Midtown                       1279
                     East Harlem                   1223
                     Chelsea                       1174
Brooklyn             Greenpoint                    1152
Manhattan            Lower East Side               1034
                     Washington Heights             941
Queens               Astoria                        940
Manhattan            West Village                   852
